# Decoding Example

This google colab compatible jupyter notebooks contains an example for how to decode data from the original [RNA SeqFISH+ paper](https://doi.org/10.1038/s41586-019-1049-y).

## Instructions
To run this notebook, you will need to change to the Julia runtime environment. To do that select from the drop down menu:

Runtime > Change runtime type

In the "Change runtime type"  prompt window, select "Julia" (not julia x.xx.xx) from the Runtime type drop-down menu. Click Save.

*note: this example was generated using data and a jupyter notebook that are freely available at the [SeqFISHSyndromeDecoding github repository](https://github.com/CaiGroup/SeqFISHSyndromeDecoding) and can be run on [google colab](https://colab.research.google.com/github/CaiGroup/SeqFISHSyndromeDecoding.jl/blob/master/example_notebook/colab/example_decode_colab.jl.ipynb)*

In [ ]:
using Pkg
Pkg.add(name="DataFrames")
Pkg.add(name="GLPK")
Pkg.add(url="https://github.com/CaiGroup/SeqFISHSyndromeDecoding.jl")
using DataFrames
using CSV
using SeqFISHSyndromeDecoding
using GLPK
using Downloads

  Activating project at `d:\SeqFISH+Processing_project\SeqFISHSyndromeDecoding\example_notebook`


This notebook shows demonstrates how to use SeqFISHSyndromeDecoding. The example data was taken from the 561 channel of cell number 8 in position 4 of replicate 2 of the 2019 SeqFISH+ NIH3T3 cell experiment. This particular subset of the data was chosen for its small size.

First load the codebook that we will use to decode our sample data.

In [ ]:
cb = DataFrame(CSV.File(Downloads.download("https://raw.githubusercontent.com/CaiGroup/SeqFISHSyndromeDecoding.jl/refs/heads/master/example_data/codebook_ch_561.csv")))
first(cb, 5)

Row,gene,block1,block2,block3,block4
,String31,Int64,Int64,Int64,Int64
1,Atp6v0e2,17,16,13,0
2,Pclo,14,14,4,4
3,Higd1a,0,1,1,0
4,Srrm1,6,14,4,16
5,Mapk8ip3,14,19,13,0


Define the [parity check matrix](https://en.wikipedia.org/wiki/Parity-check_matrix) for the codebook

In [29]:
H = [1 1 -1 -1;]

1×4 Matrix{Int64}:
 1  1  -1  -1

We can verify that H is actually the parity check matrix of the codebook.

In [30]:
all(H * Matrix(cb[:,2:end])' .% 20 .== 0)

true

Next we can load the aligned points from each hybridization for our example cell.

In [ ]:
pnts = DataFrame(CSV.File(Downloads.download("https://raw.githubusercontent.com/CaiGroup/SeqFISHSyndromeDecoding.jl/refs/heads/master/example_data/example_cell_points.csv")))
first(pnts, 5)

Row,hyb,x,y,s,w
,Int64,Float64,Float64,Float64,Float64
1,1,767.664,1463.64,1.22,633.14
2,1,759.413,1534.17,1.22,1118.99
3,1,757.458,1501.22,1.22,866.506
4,1,808.817,1400.84,1.22,1292.37
5,1,804.688,1448.16,1.22,1734.99


The SeqFISHSyndromeDecoding package requires that they hybridization column be UInt8s (to increase efficiency), and that
there be a z column (for generality to 3d data)

In [32]:
pnts.z = zeros(Float64, nrow(pnts))
pnts.hyb = UInt8.(pnts.hyb);

Next we initialize a ```DecodeParams``` object, and set the parameters

In [33]:
params = DecodeParams()

set_lat_var_cost_coeff(params, 8.0)
set_z_var_cost_coeff(params, 0.0)
set_lw_var_cost_coeff(params, 3.2)
set_s_var_cost_coeff(params, 0.0)
set_free_dot_cost(params, 1.0)

set_xy_search_radius(params, sqrt(size(H)[2]/6.0)*3)
set_z_search_radius(params, 0.0);

We can then decode

In [34]:
barcodes = decode_syndromes!(pnts, cb, H, params)
first(barcodes, 5)

5×6 DataFrame
 Row │ cpath                       cost      gene_number  x        y        z   
     │ Any                         Any       Any          Any      Any      Any 
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [2425, 8268, 10674, 12717]  1.49284   1066         756.288  1461.89  0.0
   2 │ [609, 7738, 10376, 14851]   2.51794   779          754.733  1469.27  0.0
   3 │ [884, 5309, 9951, 13425]    1.5475    1790         754.37   1475.42  0.0
   4 │ [2424, 7349, 10938, 15707]  2.62614   923          756.263  1485.94  0.0
   5 │ [86, 5513, 11990, 14538]    0.890877  593          753.649  1487.59  0.0


Row,gene,gene_number,cpath,cost,x,y,z,cc,cc_size
,String31?,Any,Any,Float64,Any,Any,Any,Int64,Int64
1,Srp68,734,"[2, 6958, 10677, 13228]",2.049,759.431,1534.0,0.0,1,1
2,Rbm28,452,"[9, 5343, 10954, 15545]",0.754996,783.107,1541.98,0.0,2,1
3,negative_control,7734,"[13, 4282, 9827, 15554]",1.63603,789.039,1464.92,0.0,3,3
4,negative_control,4556,"[2601, 7382, 9609, 13260]",2.79726,788.941,1465.46,0.0,3,3
5,Eif2b1,833,"[16, 7582, 11320, 12966]",3.32181,795.092,1522.84,0.0,4,1


Alternatively, if we aren't sure what parameters we want to use, we can save time by splitting decode_syndromes! into its two steps. First we can identify barcode candidates with the ```get_codepaths``` (named for the paths that candidate barcodes take the the decoding graph in figure 1a) function using the least strict parameter set that we are interested in.

In [35]:
candidates = get_codepaths(pnts, cb, H, params)
first(candidates, 5)

5×6 DataFrame
 Row │ cpath                       cost      gene_number  x        y        z   
     │ Any                         Any       Any          Any      Any      Any 
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [2425, 8268, 10674, 12717]  1.49284   1066         756.288  1461.89  0.0
   2 │ [609, 7738, 10376, 14851]   2.51794   779          754.733  1469.27  0.0
   3 │ [884, 5309, 9951, 13425]    1.5475    1790         754.37   1475.42  0.0
   4 │ [2424, 7349, 10938, 15707]  2.62614   923          756.263  1485.94  0.0
   5 │ [86, 5513, 11990, 14538]    0.890877  593          753.649  1487.59  0.0


Row,gene,gene_number,cpath,cost,x,y,z
,String31?,Any,Any,Any,Any,Any,Any
1,Higd1a,3,"[4049, 4233, 8660, 16526]",1.06754,990.275,1517.47,0.0
2,Srrm1,4,"[891, 6962, 9173, 15526]",2.14305,764.521,1474.04,0.0
3,Srrm1,4,"[1067, 7062, 9276, 15629]",0.241011,869.379,1540.38,0.0
4,Srrm1,4,"[1072, 7067, 9281, 15632]",1.17522,873.084,1535.57,0.0
5,Srrm1,4,"[1084, 7076, 9291, 15645]",2.8198,882.568,1479.06,0.0


In [36]:
candidates

Row,gene,gene_number,cpath,cost,x,y,z
,String31?,Any,Any,Any,Any,Any,Any
1,Higd1a,3,"[4049, 4233, 8660, 16526]",1.06754,990.275,1517.47,0.0
2,Srrm1,4,"[891, 6962, 9173, 15526]",2.14305,764.521,1474.04,0.0
3,Srrm1,4,"[1067, 7062, 9276, 15629]",0.241011,869.379,1540.38,0.0
4,Srrm1,4,"[1072, 7067, 9281, 15632]",1.17522,873.084,1535.57,0.0
5,Srrm1,4,"[1084, 7076, 9291, 15645]",2.8198,882.568,1479.06,0.0
6,Csnk2a1,6,"[3176, 7883, 10269, 13762]",2.51983,832.734,1511.71,0.0
7,Wbp11,7,"[3814, 6971, 9184, 14443]",0.845021,777.584,1569.5,0.0
8,Wbp11,7,"[3950, 7064, 9278, 14491]",0.113491,869.787,1497.33,0.0
9,Mat2a,8,"[2631, 4146, 9251, 14615]",1.6675,842.317,1441.05,0.0


We can then use the ```choose_optimal_codepaths``` function to find the same barcodew that we found earlier

In [37]:
barcodes_again = choose_optimal_codepaths(pnts, cb, H, params, candidates, GLPK.Optimizer)
barcodes == barcodes_again

true

We can now also try choosing candidates using stricter parameters. This saves computation time by reducing the number of times that we have to run ```get_codepaths```.

In [38]:
strict_params = DecodeParams()
set_lat_var_cost_coeff(strict_params, 12.0)
set_z_var_cost_coeff(strict_params, 0.0)
set_lw_var_cost_coeff(strict_params, 4.8)
set_s_var_cost_coeff(strict_params, 0.0)
set_free_dot_cost(strict_params, 1.0)

set_xy_search_radius(strict_params, sqrt(size(H)[2]/6.0)*3)
set_z_search_radius(strict_params, 0.0);


stricter_barcodes = choose_optimal_codepaths(pnts, cb, H, strict_params, candidates, GLPK.Optimizer)
first(stricter_barcodes, 5)

Row,gene,gene_number,cpath,cost,x,y,z,cc,cc_size
,String31?,Any,Any,Float64,Any,Any,Any,Int64,Int64
1,Srp68,734,"[2, 6958, 10677, 13228]",3.07349,759.431,1534.0,0.0,1,1
2,Rbm28,452,"[9, 5343, 10954, 15545]",1.13249,783.107,1541.98,0.0,2,1
3,negative_control,7734,"[13, 4282, 9827, 15554]",2.45405,789.039,1464.92,0.0,3,1
4,Rxra,1948,"[17, 8104, 10962, 14093]",1.53215,796.282,1472.55,0.0,4,1
5,Pam,108,"[18, 5567, 10473, 16161]",1.37,804.52,1448.24,0.0,5,1


We can compare the decoding results using the two different sets of parameters.

In [39]:
println("Number of gene encoding barcodes: ", sum(barcodes.gene .!= "negative_control"))
estimated_false_discovery_rate = sum(barcodes.gene .== "negative_control")*sum(cb.gene .!= "negative_control")/sum(cb.gene .== "negative_control")/sum(barcodes.gene .!= "negative_control")
println("Estimated False Discovery rate: ", estimated_false_discovery_rate)

Number of gene encoding barcodes: 1472
Estimated False Discovery rate: 0.02474342865261177


In [40]:
println("Number of gene encoding barcodes: ", sum(stricter_barcodes.gene .!= "negative_control"))
estimated_false_discovery_rate = sum(stricter_barcodes.gene .== "negative_control")*sum(cb.gene .!= "negative_control")/sum(cb.gene .== "negative_control")/sum(stricter_barcodes.gene .!= "negative_control")
println("Estimated False Discovery rate: ", estimated_false_discovery_rate)

Number of gene encoding barcodes: 1116
Estimated False Discovery rate: 0.014718418730459346


The less strict parameter set decodes about 40% more gene encoding barcodes at a cost of having twice the estimated false discovery rate. Since the estimated false positive rate is still small, it is probably an acceptable trade off.

To save your results, use the ```CSV.write``` command.

In [41]:
CSV.write("example_results.csv", barcodes)

"example_results.csv"